In [1]:
from proveit.logic import Equals, TRUE, FALSE, Not
from proveit.common import a, b, c, d, x, y, z, fx, fy, gx, gy

In [2]:
a_eq_b = Equals(a, b)
a_eq_b

In [3]:
a_eq_b.prove({a_eq_b})

The reversed form of an equation is decuded automatically via `Equals.deduceSideEffects`:

In [4]:
Equals(b, a).prove({a_eq_b})

Proofs via boolean equality are automatic via `Equals.decuceSideEffects`:

In [5]:
a.prove({Equals(a, TRUE)})

In [6]:
Not(b).prove({Equals(b, FALSE)})

Going the other direction, boolean equalities are proven automatically via `Equals.conclude`:

In [7]:
Equals(x, TRUE).prove({x})

In [8]:
Equals(TRUE, y).prove({y})

In [9]:
Equals(c, FALSE).prove({Not(c)})

In [10]:
Equals(FALSE, c).prove({Not(c)})

Transitivity chains are automatically proven via `Equals.conclude` using a bi-directional search (meeting in the middle from both ends).

In [11]:
Equals(a, d).prove([Equals(a, b), Equals(b, c), Equals(c, d)])

Derive a contradiction, where `FALSE` is implicated, via equating something with `FALSE`:

In [12]:
Equals(a, FALSE).deriveContradiction([Equals(a, FALSE)])

In [13]:
Equals(FALSE, b).deriveContradiction([Equals(b, FALSE)])

In [14]:
Equals(x, c).deriveIsInSingleton([Equals(x, c)])

AttributeError: Equals instance has no attribute 'deriveLeft'